>Get the model :

In [ ]:
rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/GATConv.1.model.single_batch.2607.pt \
/media/concha-eloko/Linux/PPT_clean/ficheros_28032023

rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/GATv2.1.model.single_batch.2707.pt \
/media/concha-eloko/Linux/PPT_clean/ficheros_28032023


In [1]:
from torch_geometric.data import HeteroData, DataLoader
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, SAGEConv, HeteroConv , GATConv, GATv2Conv
from torch_geometric.utils import negative_sampling
from torch_geometric.loader import LinkNeighborLoader

import torch
from torch import nn 
import torch.nn.functional as F
import torch.optim as optim

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder , label_binarize , OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

import os 
import pandas as pd
import numpy as np
from tqdm import tqdm
from itertools import product
import random
from collections import Counter
import warnings
warnings.filterwarnings("ignore")


In [2]:
path_work = "/media/concha-eloko/Linux/PPT_clean"

    # Open the DF
DF_info = pd.read_csv(f"{path_work}/DF_Dpo.final.2705.tsv", sep = "\t" ,  header = 0 )
    # Open the embeddings
DF_embeddings = pd.read_csv(f"{path_work}/Dpo.2705.embeddings.ultimate.csv", sep = ",", header= None )
DF_embeddings.rename(columns={0: 'index'}, inplace=True)

    # Filter the DF :
DF_info_filtered = DF_info[~DF_info["KL_type_LCA"].str.contains("\\|")]
DF_info_ToReLabel = DF_info[DF_info["KL_type_LCA"].str.contains("\\|")]
all_data = pd.merge(DF_info_filtered , DF_embeddings , on = "index")

# Mind the over representation of outbreaks :
all_data = all_data.drop_duplicates(subset = ["Infected_ancestor","index","prophage_id"] , keep = "first").reset_index(drop=True)
all_data



,Phage,KL_type_LCA,Infected_ancestor,Protein_name,Dataset,index,seq,prophage_id,1,2,...,1271,1272,1273,1274,1275,1276,1277,1278,1279,1280
0,GCF_016651625.1__phage29,KL43,GCF_016651625.1,GCF_016651625.1__phage29__142,ppt,ppt__1,MSVPNQTPYNIYTANGLTTVFTYEFYIISASDLRVSINGDVVTSGY...,prophage_12186,-0.018416,0.022387,...,0.030484,0.004437,0.087907,0.015800,0.025778,0.065790,0.034045,-0.070899,0.016068,0.065339
1,GCF_016651625.1__phage29,KL43,GCF_016651625.1,GCF_016651625.1__phage29__150,anubis,anubis__0,MRANLIKTNFTAGEISPRLMGRVDIARYANGAKIIENAVCVVQGGV...,prophage_12186,0.036016,0.005938,...,0.008465,-0.037612,0.008772,0.010556,-0.049738,-0.012549,0.092624,-0.136602,-0.191378,0.135658
2,GCF_016651625.1__phage12,KL43,GCF_016651625.1,GCF_016651625.1__phage12__59,ppt,ppt__4,MSISKRNFLKAVSCAYFFYSFKALTKVNQPIEDYISTKDKNTWPSK...,prophage_924,0.026004,0.024372,...,0.054964,-0.026018,0.018206,0.036751,-0.032549,0.064112,0.061520,-0.024423,-0.027998,0.028089
3,GCF_019928025.1__phage0,KL43,n1471,GCF_019928025.1__phage0__10,ppt,ppt__4,MSISKRNFLKAVSCAYFFYSFKALTKVNQPIEDYISTKDKNTWPSK...,prophage_2929,0.026004,0.024372,...,0.054964,-0.026018,0.018206,0.036751,-0.032549,0.064112,0.061520,-0.024423,-0.027998,0.028089
4,GCF_004313505.1__phage4,KL14,GCF_004313505.1,GCF_004313505.1__phage4__113,anubis,anubis__5,MSEYDTGNPVPSASMPDAWDNMQSIDKFVNSSDETITTRTGQQLDT...,prophage_11091,-0.023648,0.052674,...,-0.003019,-0.025991,0.068538,-0.051192,0.026481,0.069100,0.017813,-0.103797,0.018961,0.117058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9672,GCF_015209405.1__phage17,KL2,81.9/1004099,GCF_015209405.1__phage17__0,anubis,anubis__1643,MASIKELPRWEDEVYQIARGDKVEGGVGGIANMQAKTLAERTRYLK...,prophage_12,-0.014377,0.119705,...,0.040801,-0.001167,0.108840,0.008335,0.026747,0.046616,0.033648,-0.092224,0.048720,0.034757
9673,GCF_004311345.1__phage11,KL34,GCF_004311345.1,GCF_004311345.1__phage11__99,anubis,anubis__1644,MTANYPASILPPNATAVERAIDRASAAALERLPVYLIRWVKDPDSC...,prophage_11944,-0.001691,-0.067717,...,0.119783,0.073706,0.057788,0.004341,0.007389,-0.081588,0.100995,-0.045545,0.021685,0.013167
9674,GCF_900172635.1__phage3,KL124,GCF_900172635.1,GCF_900172635.1__phage3__1811,anubis,anubis__1647,MTANYPASILPPNATAVERAIDRASAAALERLPVYLIRWVKDPDSC...,prophage_11522,-0.002816,-0.055336,...,0.110956,0.056314,0.059700,0.004641,0.016676,-0.071711,0.095491,-0.055784,0.015615,0.000201
9675,GCF_013604525.1__phage18,KL30,n49324932,GCF_013604525.1__phage18__52,anubis,anubis__1649,MANIEKLGSSSPEVLLKNATNLDKLVNGRESESLPDRFGVLRKTWH...,prophage_2576,-0.000031,0.046085,...,-0.016720,-0.006530,0.077754,-0.024675,0.026408,0.016113,0.043904,-0.091418,0.078778,0.105842


In [5]:
just_chekin = all_data.drop_duplicates(subset = ["Infected_ancestor"] , keep = "first").reset_index(drop=True)


In [8]:
just_chekin[just_chekin["KL_type_LCA"] == "KL30"]

,Phage,KL_type_LCA,Infected_ancestor,Protein_name,Dataset,index,seq,prophage_id,1,2,...,1271,1272,1273,1274,1275,1276,1277,1278,1279,1280
251,GCF_902156555.1__phage3,KL30,GCF_902156555.1,GCF_902156555.1__phage3__499,anubis,anubis__12,MAYSLVQPSLAGGEISPSLYGRIDLEKYQTSLRRCRNFIVRQSGGI...,prophage_1828,0.001141,0.019932,...,0.025690,-0.063304,0.023525,0.053397,-0.015055,-0.012662,0.054703,-0.113662,-0.111309,0.195060
252,GCF_001611095.1__phage5,KL30,n49894989,GCF_001611095.1__phage5__1354,anubis,anubis__12,MAYSLVQPSLAGGEISPSLYGRIDLEKYQTSLRRCRNFIVRQSGGI...,prophage_5,0.001141,0.019932,...,0.025690,-0.063304,0.023525,0.053397,-0.015055,-0.012662,0.054703,-0.113662,-0.111309,0.195060
254,GCF_900500875.1__phage13,KL30,n4992,GCF_900500875.1__phage13__368,anubis,anubis__12,MAYSLVQPSLAGGEISPSLYGRIDLEKYQTSLRRCRNFIVRQSGGI...,prophage_5,0.001141,0.019932,...,0.025690,-0.063304,0.023525,0.053397,-0.015055,-0.012662,0.054703,-0.113662,-0.111309,0.195060
260,GCF_001701935.1__phage4,KL30,n4988,GCF_001701935.1__phage4__115,anubis,anubis__12,MAYSLVQPSLAGGEISPSLYGRIDLEKYQTSLRRCRNFIVRQSGGI...,prophage_5,0.001141,0.019932,...,0.025690,-0.063304,0.023525,0.053397,-0.015055,-0.012662,0.054703,-0.113662,-0.111309,0.195060
262,GCF_017310305.1__phage5,KL30,n4996,GCF_017310305.1__phage5__1339,anubis,anubis__12,MAYSLVQPSLAGGEISPSLYGRIDLEKYQTSLRRCRNFIVRQSGGI...,prophage_5,0.001141,0.019932,...,0.025690,-0.063304,0.023525,0.053397,-0.015055,-0.012662,0.054703,-0.113662,-0.111309,0.195060
280,GCF_017311675.1__phage26,KL30,n4998,GCF_017311675.1__phage26__12,anubis,anubis__12,MAYSLVQPSLAGGEISPSLYGRIDLEKYQTSLRRCRNFIVRQSGGI...,prophage_5,0.001141,0.019932,...,0.025690,-0.063304,0.023525,0.053397,-0.015055,-0.012662,0.054703,-0.113662,-0.111309,0.195060
897,GCF_900508005.1__phage7,KL30,81/100,GCF_900508005.1__phage7__30,ppt,ppt__105,MEQHAGDFSVTVRDVRQLTQPESDLLTLLWVLEGSVNLAVAEGASQ...,prophage_2576,-0.036591,-0.050688,...,0.049113,-0.000043,-0.078614,0.031001,-0.041555,0.050090,0.046239,-0.164150,-0.009517,0.032549
898,GCF_900506185.1__phage0,KL30,GCF_900506185.1,GCF_900506185.1__phage0__41,ppt,ppt__105,MEQHAGDFSVTVRDVRQLTQPESDLLTLLWVLEGSVNLAVAEGASQ...,prophage_13343,-0.036591,-0.050688,...,0.049113,-0.000043,-0.078614,0.031001,-0.041555,0.050090,0.046239,-0.164150,-0.009517,0.032549
1018,GCF_012271775.1__phage4,KL30,n4985,GCF_012271775.1__phage4__69,ppt,ppt__157,MQLRKLLLPGLLSVTLLSGCSLFNSEEDVVKMSPLPTVENQFTPST...,prophage_79,0.021934,0.059539,...,0.063090,-0.044104,-0.026571,-0.077920,-0.025975,-0.055033,0.150015,-0.129620,-0.068623,0.148908
1059,GCF_004803085.1__phage5,KL30,n4999,GCF_004803085.1__phage5__69,ppt,ppt__157,MQLRKLLLPGLLSVTLLSGCSLFNSEEDVVKMSPLPTVENQFTPST...,prophage_79,0.021934,0.059539,...,0.063090,-0.044104,-0.026571,-0.077920,-0.025975,-0.055033,0.150015,-0.129620,-0.068623,0.148908


In [49]:
indexation_nodes_A = all_data["Infected_ancestor"].unique().tolist()  
indexation_nodes_B1 = all_data["Phage"].unique().tolist() + [f"Dpo_to_predict_{n}" for n in DF_embeddings["index"].unique().tolist()]
indexation_nodes_B2 = DF_embeddings["index"].unique().tolist() 

ID_nodes_A = {item:index for index, item in enumerate(indexation_nodes_A)}
ID_nodes_A_r = {index:item for index, item in enumerate(indexation_nodes_A)}

ID_nodes_B1 = {item:index for index, item in enumerate(indexation_nodes_B1)}
ID_nodes_B1_r = {index:item for index, item in enumerate(indexation_nodes_B1)}

ID_nodes_B2 = {item:index for index, item in enumerate(indexation_nodes_B2)}
ID_nodes_B2_r = {index:item for index, item in enumerate(indexation_nodes_B2)}



In [ ]:
# *****************************************************************************
# Load the Dataframes :
path_work = "/media/concha-eloko/Linux/PPT_clean"
graph_data = torch.load(f'{path_work}/graph_file.2607.OHE.pt')
graph_data

In [57]:
class GNN(torch.nn.Module):
    def __init__(self, edge_type , conv, hidden_channels, heads, dropout): 
        super().__init__()
        self.conv = conv((-1,-1), hidden_channels, add_self_loops = False, heads = heads, dropout = dropout, shared_weights = True)
        self.hetero_conv = HeteroConv({edge_type: self.conv})
    def forward(self, x_dict, edge_index_dict):
        x = self.hetero_conv(x_dict, edge_index_dict)
        return x

# Classifier, Binary :
class EdgeDecoder(torch.nn.Module):
    def __init__(self, hidden_channels, heads):
        super().__init__()
        self.lin1 = torch.nn.Linear(heads*hidden_channels + 127, 512)
        self.lin2 = torch.nn.Linear(512, 1)

    def forward(self, x_dict_A , x_dict_B1, graph_data):
        edge_type = ("B1", "infects", "A")
        edge_feat_A = x_dict_A["A"][graph_data[edge_type].edge_label_index[1]]
        edge_feat_B1 = x_dict_B1["B1"][graph_data[edge_type].edge_label_index[0]]
        features_phage = torch.cat((edge_feat_A ,edge_feat_B1), dim=-1)
        x = self.lin1(features_phage).relu()
        x = self.lin2(x)
        return x.view(-1)

class Model(torch.nn.Module):
    def __init__(self, conv, hidden_channels, heads, dropout):
        super().__init__()
        self.single_layer_model = GNN(("B2", "expressed", "B1") ,conv, hidden_channels,heads,dropout)
        self.EdgeDecoder = EdgeDecoder(hidden_channels,heads)

    def forward(self, graph_data):
        b1_nodes = self.single_layer_model(graph_data.x_dict , graph_data.edge_index_dict)
        a_nodes =  graph_data.x_dict
        out = self.EdgeDecoder(a_nodes ,b1_nodes , graph_data)
        return out


In [58]:
@torch.no_grad()
def make_predictions(model, data):
    model.eval() 
    output = model(data)
    probabilities = torch.sigmoid(output)  # Convert output to probabilities
    predictions = probabilities.round()  # Convert probabilities to class labels
    return predictions, probabilities

# Load the saved model
hidden_channels = 1000
conv = GATv2Conv
heads = 1
dropout = 0.1
model = Model( conv, hidden_channels, heads, dropout)
model.load_state_dict(torch.load(f"{path_work}/ficheros_28032023/GATv2.1.model.single_batch.2707.pt"))


<All keys matched successfully>

> Make the edges match 

In [ ]:
def get_nodes_id(B1A_index_file) :
    B1A_index_file = B1A_index_file.numpy()
    B1A_index_file = tuple(zip(B1A_index_file[0],B1A_index_file[1]))
    id_file = [(ID_nodes_B1_r[tup[0]] , ID_nodes_A_r[tup[1]]) for tup in B1A_index_file]
    return id_file

In [12]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/77_strains_phage_project"
path_Dpo_f = f"{path_project}/rbp_work/77_full_Dpo"
path_Dpo_d = f"{path_project}/rbp_work/77_domain_Dpo"
path_pdb = f"{path_project}/77_phages_out"
path_Dpo_domain_org = "/media/concha-eloko/Linux/depolymerase_building/clean_77_phages_depo"

dpo_embeddings = pd.read_csv(f"{path_project}/rbp_work/Dpo_domains_77.esm2.embedding.csv", sep = "," , header = None)
dpo_embeddings = dpo_embeddings.drop([1281] , axis = 1)
dpo_embeddings.set_index([0], inplace = True)

In [11]:
# *****************************************************************************
# Load the Dataframes :
path_work = "/media/concha-eloko/Linux/PPT_clean"
graph_data = torch.load(f'{path_work}/graph_file.2607.OHE.pt')
graph_data

HeteroData(
  A={ x=[4530, 127] },
  B1={ x=[11339, 0] },
  B2={ x=[3608, 1280] },
  (B1, infects, A)={
    edge_index=[2, 7731],
    y=[7731]
  },
  (B2, expressed, B1)={
    edge_index=[2, 13285],
    y=[13285]
  },
  (A, harbors, B1)={
    edge_index=[2, 7731],
    y=[7731]
  }
)

In [72]:
pred_data = HeteroData()

# Defining the nodes :
l_dpos = len(dpo_embeddings)
pred_data["A"].x = graph_data.x_dict["A"]
pred_data["B1"].x = torch.empty((l_dpos, 0))
pred_data["B2"].x = torch.tensor(dpo_embeddings.values , dtype=torch.float)
# Defining the edge  :
edge_index_B2_B1 = torch.tensor([[i , i] for i in range(l_dpos)] , dtype=torch.long)
pred_data['B2', 'expressed', 'B1'].edge_index = edge_index_B2_B1.t().contiguous()
edge_index_B1_A = torch.tensor([[i,j] for i in range(l_dpos) for j in range(len(pred_data["A"].x))] , dtype=torch.long)
pred_data['B1', 'infects', 'A'].edge_index = graph_data['B1', 'infects', 'A'].edge_index
#pred_data['B1', 'infects', 'A'].edge_index = edge_index_B1_A.t().contiguous()
# The y files :
pred_data['B2', 'expressed', 'B1'].y = torch.ones(len(pred_data['B2', 'expressed', 'B1'].edge_index[0]))

In [73]:
pred_data

HeteroData(
  A={ x=[4530, 127] },
  B1={ x=[75, 0] },
  B2={ x=[75, 1280] },
  (B2, expressed, B1)={
    edge_index=[2, 75],
    y=[75]
  },
  (B1, infects, A)={ edge_index=[2, 7731] }
)

In [66]:
graph_data

HeteroData(
  A={ x=[4530, 127] },
  B1={ x=[11339, 0] },
  B2={ x=[3608, 1280] },
  (B1, infects, A)={
    edge_index=[2, 7731],
    y=[7731]
  },
  (B2, expressed, B1)={
    edge_index=[2, 13285],
    y=[13285]
  },
  (A, harbors, B1)={
    edge_index=[2, 7731],
    y=[7731]
  }
)

In [55]:
def get_nodes_id(B1A_index_file) :
    B1A_index_file = B1A_index_file.numpy()
    B1A_index_file = tuple(zip(B1A_index_file[0],B1A_index_file[1]))
    id_file = [(dpo_embeddings.index[tup[0]] , ID_nodes_A_r[tup[1]]) for tup in B1A_index_file]
    return id_file

test = get_nodes_id(pred_data[("B1", "infects", "A")]["edge_index"])


[('K15PH90__cds_55_Dpo_domain', 'GCF_016651625.1'),
 ('K15PH90__cds_55_Dpo_domain', 'n1471'),
 ('K15PH90__cds_55_Dpo_domain', 'GCF_004313505.1'),
 ('K15PH90__cds_55_Dpo_domain', 'n830830'),
 ('K15PH90__cds_55_Dpo_domain', 'n29132913'),
 ('K15PH90__cds_55_Dpo_domain', '81.9/1003170'),
 ('K15PH90__cds_55_Dpo_domain', 'n2940'),
 ('K15PH90__cds_55_Dpo_domain', 'n3199'),
 ('K15PH90__cds_55_Dpo_domain', 'n2949'),
 ('K15PH90__cds_55_Dpo_domain', 'n34153415'),
 ('K15PH90__cds_55_Dpo_domain', 'n3320'),
 ('K15PH90__cds_55_Dpo_domain', 'n2882'),
 ('K15PH90__cds_55_Dpo_domain', 'n3067'),
 ('K15PH90__cds_55_Dpo_domain', 'n3249'),
 ('K15PH90__cds_55_Dpo_domain', 'n2921'),
 ('K15PH90__cds_55_Dpo_domain', 'n3099'),
 ('K15PH90__cds_55_Dpo_domain', 'n28562856'),
 ('K15PH90__cds_55_Dpo_domain', 'n3242'),
 ('K15PH90__cds_55_Dpo_domain', 'n32643264'),
 ('K15PH90__cds_55_Dpo_domain', 'n29292929'),
 ('K15PH90__cds_55_Dpo_domain', 'n32403240'),
 ('K15PH90__cds_55_Dpo_domain', 'n34723472'),
 ('K15PH90__cds_55_

In [87]:
wow = 0
with open(f"{path_work}/77_Dpos.PPT_pred.80.tsv", "w") as outfile :
    l_dpos = len(dpo_embeddings)
    # Defining the nodes : 
    pred_data["A"].x = graph_data.x_dict["A"]
    pred_data["B1"].x = torch.empty((l_dpos, 0))
    pred_data["B2"].x = torch.tensor(dpo_embeddings.values , dtype=torch.float)
    # Defining the edges : 
    edge_index_B2_B1 = torch.tensor([[i , i] for i in range(l_dpos)] , dtype=torch.long)
    pred_data[('B2', 'expressed', 'B1')].edge_index = edge_index_B2_B1.t().contiguous()
    for dpo_index in range(l_dpos) :
        edge_index_B1_A = torch.tensor([[dpo_index,j] for j in range(len(pred_data["A"].x))] , dtype=torch.long)
        pred_data[('B1', 'infects', 'A')]["edge_label_index"] = edge_index_B1_A.t().contiguous()
        predictions, probabilities = make_predictions(model, pred_data)
        test = get_nodes_id(pred_data[("B1", "infects", "A")]["edge_label_index"])
        results = tuple(zip(test,predictions.numpy(),probabilities.numpy()))
        results_phage = []
        for link in results : 
            KLtype = all_data[all_data["Infected_ancestor"] == link[0][1]]["KL_type_LCA"].values[0]
            line = (link[0][0],KLtype, link[0][1], link[1],link[2])
            results_phage.append(line)
        df = pd.DataFrame(results_phage, columns=['Dpo_index', 'KLtype','Ancestor','Prediction','Score'])
        df = df[df["Score"] >= 0.80]
        kltype_res = Counter(df["KLtype"])
        outfile.write(f"{link[0][0]}\t{kltype_res}\n")

In [84]:
predictions

tensor([0., 0., 0.,  ..., 0., 0., 1.])

In [80]:
wow

0

In [40]:
with open(f"{path_work}/results.2707.95.tsv", "w") as outfile : 
    for phage in tqdm(ID_nodes_B1) : 
        if phage.count("Dpo_to_predict_") > 0 :
            edge_index_B1_A_jugement = []
            phage_id = ID_nodes_B1[phage]
            for strain_id in ID_nodes_A_r : 
                edge_index_B1_A_jugement.append((phage_id , strain_id))
            edge_index_B1_A_jugement = torch.tensor(edge_index_B1_A_jugement , dtype=torch.long)
            graph_data[("B1", "infects", "A")]["edge_label_index"] = edge_index_B1_A_jugement.t().contiguous()
            predictions, probabilities = make_predictions(model, graph_data)
            test = get_nodes_id(graph_data[("B1", "infects", "A")]["edge_label_index"])
            results = tuple(zip(test,predictions.numpy(),probabilities.numpy()))
            results_phage = []
            for link in results : 
                KLtype = all_data[all_data["Infected_ancestor"] == link[0][1]]["KL_type_LCA"].values[0]
                line = (link[0][0].split("to_predict_")[1],KLtype, link[0][1], link[1],link[2])
                results_phage.append(line)
            df = pd.DataFrame(results_phage, columns=['Dpo_index', 'KLtype','Ancestor','Prediction','Score'])
            df = df[df["Score"] >= 0.95]
            kltype_res = Counter(df["KLtype"])
            outfile.write(f"{link[0][0].split('to_predict_')[1]}\t{kltype_res}\n")

339750

In [42]:
pred_data

HeteroData(
  A={ x=[4530, 127] },
  B1={ x=[75, 0] },
  B2={ x=[75, 1280] },
  (B2, expressed, B1)={ edge_index=[2, 75] },
  (B1, infects, A)={ edge_index=[2, 339750] }
)

In [ ]:
# Make edge file
# Node B1 (prophage) - Node A (bacteria) :
edge_index_B1_A = set()
for _, row in all_data.iterrows():
    edge_index_B1_A.add((ID_nodes_B1[row["Phage"]], ID_nodes_A[row["Infected_ancestor"]]))
edge_index_B1_A = list(edge_index_B1_A)  
edge_index_B1_A = torch.tensor(edge_index_B1_A , dtype=torch.long)

# Node A (bacteria) - Node B1 (prophage) :
edge_index_A_B1 = set()
for _, row in all_data.iterrows() :
    edge_index_A_B1.add((ID_nodes_A[row["Infected_ancestor"]], ID_nodes_B1[row["Phage"]]))
edge_index_A_B1 = list(edge_index_A_B1)
edge_index_A_B1 = torch.tensor(edge_index_A_B1 , dtype=torch.long)

# Node B2 (depolymerase) - Node B1 (prophage) :
edge_index_B2_B1 = []
for phage in all_data.Phage.unique():
    all_data_phage = all_data[all_data["Phage"] == phage]
    for _, row in all_data_phage.iterrows() :
        edge_index_B2_B1.append([ID_nodes_B2[row["index"]], ID_nodes_B1[row["Phage"]]])
# Add in there the edges between the fake prophages and the each Dpos :
for prophage , index in ID_nodes_B1.items():
    if prophage.count("Dpo_to_predict_") > 0 : 
        id_dpo = prophage.split("Dpo_to_predict_")[1]
        edge_index_B2_B1.append([ID_nodes_B2[id_dpo], index])
edge_index_B2_B1 = torch.tensor(edge_index_B2_B1 , dtype=torch.long)

# feed the graph
graph_data['B1', 'infects', 'A'].edge_index = edge_index_B1_A.t().contiguous()
graph_data['B2', 'expressed', 'B1'].edge_index = edge_index_B2_B1.t().contiguous()